## **Trabajo Práctico Individual n° 2: Aprendizaje Supervisado**
____
__Universidad Tecnológica Nacional, Buenos Aires__\
__Ingeniería Industrial__\
__Ciencia de Datos - Curso I5521__
____

- Nombre: Cristian
- Apellido: Pérez Maturana
- Legajo: 147903-9

En este trabajo práctico aplicaremos algunas técnicas aprendidas para el aprendizaje automático supervisado.

Utilizaremos un dataset de airbnb donde buscaremos predecir el precio de las noches de cada hogar en funcion de algunas variables.

#### Variables:
- Unnamed: 0: Variable de contador de registros (no será utilizada en nuetro análisis)
- neighbourhood: Barrio donde se encuentra ubicado la casa.
- room_type: Tipo de hogar en alquiler.
- price: Precio por noche.
- minimum_nights: Cantidad minimas de noches que pueden alquilar el hogar.
- number_of_reviews: Cantidad de reviews
- reviews_per_month: Tasa de reviews que tienen por mes
- calculated_host_listings_count: Cantidad de hogares ofrecidos por el host.
- availability_365: Cantidad de dias que se encuentra en alquiler el hogar

## **Objetivos del trabajo practico**
- resolver los requisitos que se detallan en las proximas celdas
- dejar comentarios # en cada linea de codigo que se agregue
- fecha limite de entrega: 13 de Octubre 2022 a las 23h59

In [1]:
## se importan librerías de manipulación de datos
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt

In [2]:
## se importan librerías de aprendizaje automático
from sklearn import preprocessing
from sklearn import linear_model
from sklearn.model_selection import train_test_split, cross_val_score
from sklearn.metrics import r2_score, mean_squared_error
from sklearn.utils import shuffle
from sklearn.preprocessing import StandardScaler
from sklearn import preprocessing
from sklearn.preprocessing import LabelEncoder
from sklearn.linear_model import LinearRegression
from sklearn.svm import SVR, LinearSVR
from sklearn.metrics import mean_squared_error, make_scorer
from sklearn.model_selection import GridSearchCV

##### **Objetivo 1**
Lo primero que deberan hacer será importar
- el dataset bajo en nombre "london"
- corroborar que lo hayamos cargado correctamente.

In [3]:
# lectura de dataset
london = pd.read_csv(r'D:\CPM\Ciencia de datos\Directorio\TP 2\TP2_airbnb_london.csv')

In [4]:
# se visualizan 5 filas de muestra del dataset con todas sus features
london.tail()

,Unnamed: 0,neighbourhood,room_type,price,minimum_nights,number_of_reviews,reviews_per_month,calculated_host_listings_count,availability_365
51822,74816,Brent,Entire home/apt,60,5,1,1.0,1,7
51823,74927,Lewisham,Entire home/apt,30,2,1,1.0,1,26
51824,75003,Westminster,Entire home/apt,280,3,1,1.0,14,352
51825,75054,Tower Hamlets,Private room,35,2,1,1.0,1,13
51826,75192,Westminster,Private room,66,2,1,1.0,1,140


##### **Objetivo 2**
Ahora imprimir también las dimensiones del dataset y eliminar la variable "Unnamed:0" del dataset.
Volver a guardar el dataset sin esta variable bajo el nombre "london" 

In [5]:
# se imprimen dimensiones iniciales
london.shape

(51827, 9)

In [6]:
# eliminación de variable Unnamed: 0
london = london.drop(['Unnamed: 0'], axis=1)

In [7]:
# se verifican nuevas dimensiones
london.shape

(51827, 8)

In [8]:
# celda de código auxiliar para dropear el 95% de las samples del dataset solo a fin de probar y correr la notebook más rápido.
# se deberá saltear para correrla entera
london = london.drop(london.index[2000:51000])
london.shape

##### **Objetivo 3**
Corroborar que el dataset no cuenta con valores nulos, si los hubiera llenarlos todos con 0.

In [10]:
# se suman la cantidad de nulos por cada feature del dataset
london.isnull().sum()

neighbourhood                     0
room_type                         0
price                             0
minimum_nights                    0
number_of_reviews                 0
reviews_per_month                 0
calculated_host_listings_count    0
availability_365                  0
dtype: int64

##### **Objetivo 4**
Mostrar el tipo de dato para las variables "neighbourhood" y "room_type".

In [11]:
# con pd.info() se observan los tipos de dato tanto de dichas variables como del resto para aprovechar a verificarlas
london.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 2827 entries, 0 to 51826
Data columns (total 8 columns):
 #   Column                          Non-Null Count  Dtype  
---  ------                          --------------  -----  
 0   neighbourhood                   2827 non-null   object 
 1   room_type                       2827 non-null   object 
 2   price                           2827 non-null   int64  
 3   minimum_nights                  2827 non-null   int64  
 4   number_of_reviews               2827 non-null   int64  
 5   reviews_per_month               2827 non-null   float64
 6   calculated_host_listings_count  2827 non-null   int64  
 7   availability_365                2827 non-null   int64  
dtypes: float64(1), int64(5), object(2)
memory usage: 198.8+ KB


##### **Objetivo 5**
Generar las variables dummies para estas dos variables categóricas y unirlas al dataset london.


In [12]:
# se generan dummies para la feature neighbourhood, dentro de la variable neigh_dummies
neigh_dummies = pd.get_dummies(data=london, columns=['neighbourhood'])

Imprimir las dimensiones y primeras lineas de estas dummies

In [13]:
# dimensiones de las dummies generadas
neigh_dummies.shape

(2827, 40)

In [14]:
# primeras filas del dataframe creado
neigh_dummies.head()

,room_type,price,minimum_nights,number_of_reviews,reviews_per_month,calculated_host_listings_count,availability_365,neighbourhood_Barking and Dagenham,neighbourhood_Barnet,neighbourhood_Bexley,...,neighbourhood_Merton,neighbourhood_Newham,neighbourhood_Redbridge,neighbourhood_Richmond upon Thames,neighbourhood_Southwark,neighbourhood_Sutton,neighbourhood_Tower Hamlets,neighbourhood_Waltham Forest,neighbourhood_Wandsworth,neighbourhood_Westminster
0,Private room,35,1,131,1.71,4,262,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,Private room,70,2,2,0.07,6,62,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,Private room,45,1,14,0.14,2,364,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,Entire home/apt,102,5,7,0.07,56,295,0,0,0,...,0,0,0,0,0,0,1,0,0,0
4,Private room,29,5,114,1.13,3,285,0,1,0,...,0,0,0,0,0,0,0,0,0,0


In [15]:
# se generan dummies para la feature room_type, dentro de la variable room_dummies
room_dummies = pd.get_dummies(data=london, columns=['room_type'])

Imprimir las dimensiones y primeras lineas de estas dummies

In [16]:
# dimensiones de las dummies generadas
room_dummies.shape

(2827, 10)

In [17]:
# primeras filas del dataframe creado
room_dummies.head()

,neighbourhood,price,minimum_nights,number_of_reviews,reviews_per_month,calculated_host_listings_count,availability_365,room_type_Entire home/apt,room_type_Private room,room_type_Shared room
0,Haringey,35,1,131,1.71,4,262,0,1,0
1,Ealing,70,2,2,0.07,6,62,0,1,0
2,Islington,45,1,14,0.14,2,364,0,1,0
3,Tower Hamlets,102,5,7,0.07,56,295,1,0,0
4,Barnet,29,5,114,1.13,3,285,0,1,0


Joinear las variables dummies al dataset



In [18]:
# dado que las dummies se crearon a la par del dataset original, 
# se eliminan features originales de ambos dataframes para conservar solo las columnas binarias
neigh_dummies.drop(neigh_dummies.iloc[:, 0:7], axis=1, inplace=True)
room_dummies.drop(room_dummies.iloc[:, 0:7], axis=1, inplace=True)

In [19]:
# se joinean ambas dummies generadas al dataset original y se visualizan primeras líneas
london = london.join([neigh_dummies, room_dummies])
london.head()

,neighbourhood,room_type,price,minimum_nights,number_of_reviews,reviews_per_month,calculated_host_listings_count,availability_365,neighbourhood_Barking and Dagenham,neighbourhood_Barnet,...,neighbourhood_Richmond upon Thames,neighbourhood_Southwark,neighbourhood_Sutton,neighbourhood_Tower Hamlets,neighbourhood_Waltham Forest,neighbourhood_Wandsworth,neighbourhood_Westminster,room_type_Entire home/apt,room_type_Private room,room_type_Shared room
0,Haringey,Private room,35,1,131,1.71,4,262,0,0,...,0,0,0,0,0,0,0,0,1,0
1,Ealing,Private room,70,2,2,0.07,6,62,0,0,...,0,0,0,0,0,0,0,0,1,0
2,Islington,Private room,45,1,14,0.14,2,364,0,0,...,0,0,0,0,0,0,0,0,1,0
3,Tower Hamlets,Entire home/apt,102,5,7,0.07,56,295,0,0,...,0,0,0,1,0,0,0,1,0,0
4,Barnet,Private room,29,5,114,1.13,3,285,0,1,...,0,0,0,0,0,0,0,0,1,0


##### Dividiremos nuestro dataset en la variable dependiente "y" (label o etiqueta) y las variables independientes "x"

In [20]:
# creación de la variable y, filtrando solo los precios al conformar las etiquetas del modelo
y = np.array(london[["price"]])
# se define X dropeando las etiquetas y las columnas originales por las cuales se crearon las dummies
x = london.drop(['price', 'neighbourhood','room_type'], axis=1)

##### **Objetivo 7**

Dividir el dataset entre Train y Test, tomar un test_size del 30%.

In [21]:
# partición del (x, y) en train y test, con 70-30% de relacion respectivamente, y se asigna seed arbitraria
xtrain, xtest, ytrain, ytest = train_test_split(x, y, test_size=0.30, random_state=7)

In [22]:
# se separan dummies del xtrain para poder escalar valores continuos originales
xtrain_dummies = xtrain.drop(xtrain.iloc[:, 0:5], axis=1)
# se guardan valores a escalar del xtrain
xtrain_sindummies = xtrain.drop(xtrain.iloc[:, 5:42], axis=1)

In [23]:
# se separan dummies del xtest para escalar valores continuos originales
xtest_dummies = xtest.drop(xtest.iloc[:, 0:5], axis=1)
# se guardan valores a escalar del xtest
xtest_sindummies = xtest.drop(xtest.iloc[:, 5:42], axis=1)

##### **Objetivo 8**
Con el StandardScaler generado en la linea siguiente, realizar el transform para el xtrain y xtest guardandolos en nuevas variables llamadas "xtrain_scal" y "xtest_scal".


In [24]:
# se define standard scaler ajustándolo según datos del xtrain
scaler = preprocessing.StandardScaler().fit(xtrain_sindummies)

In [25]:
# transform a los datos del xtrain y se guarda en variable
xtrain_scal = scaler.transform(xtrain_sindummies)

In [26]:
# transform a los datos del xtest y se guarda en variable
xtest_scal = scaler.transform(xtest_sindummies)

In [27]:
# se convierten dummies de xtrain en array
xtrain_dummies = xtrain_dummies.to_numpy()
# idem xtest
xtest_dummies = xtest_dummies.to_numpy()

In [28]:
# se joinean dummies a xtrain y xtest ya escalados
xtrain_london = np.concatenate((xtrain_scal,xtrain_dummies), axis=1)
xtest_london = np.concatenate((xtest_scal,xtest_dummies), axis=1)

##### **Objetivo 9**
Crear un modelo lineal y entrenarlo para nuestro dataset.

Obtener las predicciones y el "mean_squared_error" para el test set.

In [29]:
# se crea modelo de regresión lineal
lr = LinearRegression()

In [30]:
# se ajusta modelo utilizando set de entrenamiento
lr.fit(xtrain_london, ytrain)

LinearRegression()

In [31]:
# se obtienen predicciones que realiza el modelo con las muestras de test, y se guardan en 'ypred'
ypred = lr.predict(xtest_london)

In [32]:
# calculo del error del modelo comparando predicciones con ytest
np.sqrt(mean_squared_error(ytest, ypred))

36.329418196709796

##### **Objetivo 10**
Crear un SVM Regressor y un GridSearch para realizar la búsqueda de hiperparámetros.

Asignar dos posibles valores para el parámetro "kernel", y tres posibles valores para "C" y "gamma".

Setear al parámetro "refit" en True y hacer 5 CrossValidation.

In [33]:
# se define diccionario de parámetros para pasarle al grid search
param_svreg = {'kernel':('sigmoid', 'rbf'),
               'C':[10, 50, 100 ],
               'gamma':[ 0.01, 0.1, 1]}

In [34]:
# gridsearch cross-validation utilizando Support vector regressor de estimador + el resto de hiperparámetros recién definidos
regressor_svr = GridSearchCV(SVR(), param_svreg, cv=5, refit=True, verbose=3)

In [35]:
# se entrena modelo
regressor_svr.fit(xtrain_london, ytrain.ravel())

Fitting 5 folds for each of 18 candidates, totalling 90 fits
[CV 1/5] END ..C=10, gamma=0.01, kernel=sigmoid;, score=0.369 total time=   0.3s
[CV 2/5] END ..C=10, gamma=0.01, kernel=sigmoid;, score=0.404 total time=   0.2s
[CV 3/5] END ..C=10, gamma=0.01, kernel=sigmoid;, score=0.366 total time=   0.2s
[CV 4/5] END ..C=10, gamma=0.01, kernel=sigmoid;, score=0.374 total time=   0.2s
[CV 5/5] END ..C=10, gamma=0.01, kernel=sigmoid;, score=0.334 total time=   0.2s
[CV 1/5] END ......C=10, gamma=0.01, kernel=rbf;, score=0.413 total time=   0.3s
[CV 2/5] END ......C=10, gamma=0.01, kernel=rbf;, score=0.454 total time=   0.3s
[CV 3/5] END ......C=10, gamma=0.01, kernel=rbf;, score=0.425 total time=   0.3s
[CV 4/5] END ......C=10, gamma=0.01, kernel=rbf;, score=0.429 total time=   0.4s
[CV 5/5] END ......C=10, gamma=0.01, kernel=rbf;, score=0.379 total time=   0.3s
[CV 1/5] END ..C=10, gamma=0.1, kernel=sigmoid;, score=-4.061 total time=   0.3s
[CV 2/5] END ..C=10, gamma=0.1, kernel=sigmoid;,

GridSearchCV(cv=5, estimator=SVR(),
             param_grid={'C': [10, 50, 100], 'gamma': [0.01, 0.1, 1],
                         'kernel': ('sigmoid', 'rbf')},
             verbose=3)

##### **Objetivo 11**

Mostrar cuales fueron los mejores hiperparámetros y para estos obtener el mean_squared_error en el test set.

In [36]:
# se imprimen mejores hiperparámetros
regressor_svr.best_params_

{'C': 50, 'gamma': 0.1, 'kernel': 'rbf'}

In [37]:
# se imprime mejor r2
regressor_svr.best_score_

0.5095446303886344

In [38]:
# se le pasa xtest al modelo para arrojar primeras predicciones
ypred = regressor_svr.predict(xtest_london)
ypred

array([101.82713496, 117.87860548, 114.30661806,  40.29634324,
       110.62367423,  51.00610664,  47.22000522,  40.16872134,
        42.38111565,  90.87587982,  45.42952764,  40.79371976,
        93.84432965,  42.96020118,  85.32909544,  97.41048633,
        33.6519221 ,  43.62779508,  99.20787298,  45.18703053,
        40.77701813, 108.35863022, 132.76810896, 136.88968301,
        48.93443573,  50.65423825, 101.6969047 ,  37.91631705,
        59.40514434, 128.46506651, 120.47297102, 100.92580399,
        92.57307279,  69.63332468, 163.25371184, 122.41711682,
        31.70388544,  89.86514539, 103.51941694,  46.95515879,
        37.40521192,  40.0144364 ,  32.43615349, 136.38506512,
       109.08298029, 127.5342014 , 112.89121384,  92.33851668,
        94.62866216,  40.040553  ,  82.92751868, 129.96143653,
       111.82035431,  41.69083391, 137.76404946,  95.2753469 ,
       135.75065334, 109.7215336 ,  48.08926297,  36.8095383 ,
        90.85477689, 118.65943274,  50.32877827, 111.84

In [39]:
# MSE del modelo comparando predicciones con ytest
np.sqrt(mean_squared_error(ytest, ypred))

36.99809438509309

##### **Objetivo 12**
Repetir el objetivo 10 pero ahora seteando nuevos valores para los hiperparámetros mencionados con el objetivo de mejorar las predicciones.

Cuales son los mejores hiperparámetros ahora?
Obtenemos una mejora en las predicciones?
Mostrar los resultados.

In [40]:
# se define nuevo diccionario de parámetros para pasar al gridsearchCV, probando en un rango cercano a los mejores hiperparam.
# definidos en el primer gridsearch, y cambiando el kernel sigmoide por uno polinomial
param_svreg2 = { 'kernel': ('rbf', 'poly'),
                     'C': [50, 100, 200],
                 'gamma': [0.05, 0.1, 0.5]}

In [41]:
# nuevo gridsearch cross-validation utilizando Support vector regressor de estimador + hiperparámetros definidos en ultima celda
regressor_svr2 = GridSearchCV(SVR(), param_svreg2, cv=5, refit=True, verbose=3)

In [42]:
# se entrena modelo
regressor_svr2.fit(xtrain_london, ytrain.ravel())

Fitting 5 folds for each of 18 candidates, totalling 90 fits
[CV 1/5] END ......C=50, gamma=0.05, kernel=rbf;, score=0.503 total time=   0.3s
[CV 2/5] END ......C=50, gamma=0.05, kernel=rbf;, score=0.519 total time=   0.3s
[CV 3/5] END ......C=50, gamma=0.05, kernel=rbf;, score=0.520 total time=   0.3s
[CV 4/5] END ......C=50, gamma=0.05, kernel=rbf;, score=0.532 total time=   0.3s
[CV 5/5] END ......C=50, gamma=0.05, kernel=rbf;, score=0.478 total time=   0.3s
[CV 1/5] END .....C=50, gamma=0.05, kernel=poly;, score=0.074 total time=   0.1s
[CV 2/5] END .....C=50, gamma=0.05, kernel=poly;, score=0.255 total time=   0.2s
[CV 3/5] END .....C=50, gamma=0.05, kernel=poly;, score=0.204 total time=   0.2s
[CV 4/5] END .....C=50, gamma=0.05, kernel=poly;, score=0.216 total time=   0.2s
[CV 5/5] END .....C=50, gamma=0.05, kernel=poly;, score=0.232 total time=   0.2s
[CV 1/5] END .......C=50, gamma=0.1, kernel=rbf;, score=0.504 total time=   0.3s
[CV 2/5] END .......C=50, gamma=0.1, kernel=rbf;

GridSearchCV(cv=5, estimator=SVR(),
             param_grid={'C': [50, 100, 200], 'gamma': [0.05, 0.1, 0.5],
                         'kernel': ('rbf', 'poly')},
             verbose=3)

In [43]:
# se imprimen mejores hiperparámetros, en el que el mejor kernel sigue siendo el RBF, con un gamma y C que toman nuevos valores
regressor_svr2.best_params_

{'C': 200, 'gamma': 0.05, 'kernel': 'rbf'}

In [44]:
# se imprime r2 obtenido con los mismos
regressor_svr2.best_score_

0.5141935566532634

In [45]:
# se le pasa xtest al nuevo modelo para arrojar primeras predicciones
ypred2 = regressor_svr2.predict(xtest_london)
ypred2

array([101.54157333, 124.82438978, 114.79323044,  39.98987594,
       110.53598493,  51.18290197,  47.42793914,  40.50676559,
        43.56333379,  91.01973472,  45.0832289 ,  40.0987984 ,
        86.39948796,  42.27629123,  83.26766634,  96.16388047,
        28.36867123,  43.41694694,  97.35477546,  46.35723083,
        40.89804694, 111.00400825, 136.6287249 , 131.76213922,
        48.60694505,  50.58745891, 100.05106744,  38.23687236,
        58.82620373, 129.83062714, 124.05011908, 101.32806143,
        96.09159559,  73.02589274, 171.00751851, 123.2025253 ,
        31.31670617,  90.02548643, 101.85128576,  47.05876598,
        36.61081439,  39.81390306,  29.29612013, 143.3590543 ,
       109.51040757, 133.69682955, 111.87358031,  91.93362677,
        96.4288134 ,  39.66978889,  81.14513081, 129.088039  ,
       112.99792537,  44.19374909, 138.29305199,  96.27986134,
       135.49186863, 112.52122636,  45.31561268,  30.58045316,
        92.71493091, 117.16198817,  53.23344029, 111.02

In [46]:
# MSE del nuevo modelo comparando predicciones con ytest, el cual resulta prácticamente igual al del primer SVR para este
# orden de magnitud (considerando también un MSE de 36,33 en la regresión lineal genérica creada en primera instancia)
# (0,2% de variación entre mejores modelos de 1er y 2do SVR)
np.sqrt(mean_squared_error(ytest, ypred2))

37.095720074877974

In [47]:
# Teniendo en cuenta estos MSE, se podría decir que las predicciones apenas mejoran del primer juego de hiperparámetros
# a los del segundo, ya que el r2 pasa de 0.5095 a 0.5142, valor que en ambos casos sigue siendo bajo como para esperar del
# modelo una buena correlación lineal entre los resultados para estos datos de entrenamiento y futuras muestras no vistas.

In [48]:
# Aclaración: 
# al principio se dropean el 95% de las muestras ya que en las primeras pruebas los modelos no entrenaban ni durante toda la
# noche, y aun probando con un 10% del dataset, todavía le costaba mucho pasar por ciertos juegos de parámetros, que después
# terminaban resultando en scores negativos.
# Con algunos intentos más y filtrando cada vez menos samples, quizás se puedan mejorar los resultados, pero a modo de ejemplo
# para la entrega guardo la notebook así.